In [0]:


base = "abfss://lakehouse@goodreadsreviews60105179.dfs.core.windows.net/gold"
input_path  = f"{base}/features_v1"    
output_base = f"{base}/features_v2"     


In [0]:
"""
Install all required Python libraries for text processing and feature engineering. 
Includes NLTK for sentiment analysis, Sentence Transformers for embeddings, 
TextStat for readability metrics, and TextBlob for subjectivity analysis.
"""
%pip install nltk
%pip install sentence-transformers
%pip install textstat
%pip install textblob
%pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/803.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0

In [0]:
"""
Import all required libraries for Spark, text processing, and feature engineering. 
Includes Spark SQL and ML components for data manipulation and TF-IDF, 
NLTK, TextBlob, TextStat, and Sentence Transformers for NLP tasks, 
and NumPy, Pandas, and scikit-learn for data analysis and vectorization.
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, year, month, dayofweek, to_timestamp,
    length, size, split, udf
)

from pyspark.sql.types import (
    StringType, DoubleType, FloatType, ArrayType
)
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, IDF
)
from pyspark.ml import Pipeline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textstat
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import re
import re, string, emoji
from pyspark.sql.functions import udf, col, length
from pyspark.sql.types import StringType
from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
# Load cleaned Gold dataset from Lab 3 (features_v1)
df = spark.read.format("delta").load(input_path)

# Quick preview to confirm it's loaded
display(df.limit(5))
df.printSchema()





book_id,review_id,title,author_id,author_name,user_id,rating,review_text,language_code,n_votes,date_added,review_length,review_length_in_words,average_rating,number_of_reviews
7663760,fa241e939d2218940d17cff6fc30bca4,fooling some of the people all of the time: a long short (and now complete) story,1397426,david einhorn,0c5b36407771dfd65acdd812ecd51705,4.0,"investing is an obsession and not a job.i think few exemplify that more than david einhorn, one of the smartest fundamental investors in the current era. in a world, where short-sellers are castigated as profiteers and trouble makers, mr einhorn demonstrates through his most public short that fundamental short-selling is the only tool to expose fraud and corruption in financial markets.his appeal to regulators to close loopholes that allow companies to feed off tax-payer dollars is genuine but it's difficult to be an optimist when incentives are so perverse despite all the outrage in the aftermath of the financial crisis.all i can say is that mr einhorn and his ilk, continue to be the de-facto regulators of a broken financial system and if they profit from that, well isn't it just right when you see the breadth of the work they put into doing so.",eng,2071,2015-04-07,860,145,3.125,8
22387890,2628b9004c4b6710aa0c20ce094da2c1,deep water,5341919,coral moore,b78a9143ca2f0c4c7361694dc6cb0500,3.0,i liked this story despite not being entirely sold on the attraction between mario and jordan which result pretty much in insta-love. but it was still a nice short and sweet story.,eng,30,2014-06-02,180,32,3.5,2
1096390,ec6f00823d47459dc2c70fea9559c605,the uncommon reader,11781,alan bennett,c3fa377cfc84401747630b98f92758c9,4.0,cute story with lots of british humor on the perils of reading too much. like that could ever happen!,eng,17355,2010-01-05,101,19,3.9038031319910513,447
13166894,e995dd91ce98a48306e5be8d15f97c8d,death at seaworld: shamu and the dark side of killer whales in captivity,6435477,david kirby,ddc44923909c38b4d149a38431105943,4.0,"this was a good book with a very important message that everybody should at least be aware of. while i agree with a lot of the position this book took,i do have a few notes: - it clearly had an agenda, which is okay, but even though i share the same position on whales in captivity, it is quite biased and only gives a small glimpse into the other side of the debate. - it get's extremely repetitive. there were points where i had to check to make sure i hadn't lost my place because it felt like i had just read the exact same thing in previous chapters. - personally, while i was completely interested (especially being that orcas are my favourite animal), there were points, particularly when discussing the legal battle issues where i started to glaze over a little bit and speed read to get through it. all in all though, this was a very thought-provoking read that i overall very much enjoyed.",eng,15,2013-07-10,907,166,4.0,1
25430624,d6120da6fdef79aaa9e11c623e4b9559,abc dream,8108153,kim krans,419dfd723edeb5e27f50aa2382f9aa86,5.0,"concept: 5 stars art: 5 stars alphabet book with marvelous illustrations that include various items for each letter. in each illustration, readers discern all the words they can that begin with the featured letter, then check their answers against the list in the back of the book. fun, interactive way to learn the alphabet.",,122,2017-01-02,329,54,4.545454545454546,11


root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_length_in_words: integer (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- number_of_reviews: long (nullable = true)



# Splitting the dataset 

In [0]:
"""
Split the dataset into training, validation, and test subsets using a 70/15/15 ratio. 
Count the number of samples in each split and calculate the percentage distribution 
to confirm the split proportions.
"""

train, val, test = df.randomSplit([0.7, 0.15, 0.15], seed=42)

train_count = train.count()
val_count = val.count()
test_count = test.count()

total = train_count + val_count + test_count

train_pct = (train_count / total) * 100
val_pct = (val_count / total) * 100
test_pct = (test_count / total) * 100

print(f"Train: {train_pct:.2f}% | Validation: {val_pct:.2f}% | Test: {test_pct:.2f}%")


Train: 70.02% | Validation: 14.98% | Test: 15.00%


In [0]:
"""
Save the train, validation, and test DataFrames to the Gold layer in Delta format. 
Each split is stored in a separate folder for consistent access and reuse 
during model training and evaluation.
"""

train.write.format("delta").mode("overwrite").save(
   output_base + "/train/"
)

val.write.format("delta").mode("overwrite").save(
    output_base + "/validation/"
)

test.write.format("delta").mode("overwrite").save(
    output_base + "/test/"
  
)


In [0]:
"""
Load the training dataset from the Gold layer (feature_v2/train) stored in Delta format. 
Display the first ten records to verify successful loading and data integrity.
"""

train = spark.read.format("delta").load(
    output_base + "/train/"
)

display(train.limit(10))

book_id,review_id,title,author_id,author_name,user_id,rating,review_text,language_code,n_votes,date_added,review_length,review_length_in_words,average_rating,number_of_reviews
10000600,0762d8aa5cc57bd577c9230ebd244426,"old habits (wicked lovely, #2.6)",175855,melissa marr,172475fb7fcc11c5b19bf23afd7d2a16,4.0,"this takes place after ink exchange also. it focuses on niall's and irial's relationship. niall is dealing with being the new dark king. even though he doesn't desire the role, he cannot think of anyone else who is more capable of being it(besides irial who refuses it back). as niall is settling into his new role as dark king, he still struggles with his feelings for irial and leslie. irial is working behind niall's back to help keep him safe. even though he isn't the king anymore, he is willing to put himself at risk to protect niall. he is trying to take part of all the really bad things in court, to help niall transition. niall doesn't want irial to interfere and go against his commands. even though, i feel he appreciates his help. he cannot afford to have anyone go against him, if he is to truly take charge of the dark court. devlin, is the high court queen's sister. he does her bidding. even though she knows one day he will become her enemy. his true desires is with the dark court. irial has a 'friendship' with the high court queen. irial and him have had an intimate relationship from long ago. they are both struggling with their feeling for each other. i personally, think they should end up together. i feel that they make a better pairing than with leslie. though, most faeries live a polyamorous life.",en-US,1628,2013-02-09,1334,241,4.3,20
10000600,0d4b4c6eb8e4d9743fd70221bee5283b,"old habits (wicked lovely, #2.6)",175855,melissa marr,9ed7aee625cb20426b58cf5c4e660a83,5.0,"love melissa marrs wicked lovely books and this novella did not let me down! very interesting and compelling story. we learn so much more of the inner workings of the fairie regents. like irial and sorcha's close yet indiffent relationship. sorchas relationship with her brother, devlin. and my favorite the sorid love/ hate relationship of nial and irial. two of my favorite characters together again in this short but must read. couldnt put the story down i read it twice. i love nial and irial that much!",en-US,1628,2011-05-07,507,87,4.3,20
10000600,12e678568a0be603269e7c95f6389b23,"old habits (wicked lovely, #2.6)",175855,melissa marr,a51098194509966df9bd1fcca077f3b9,4.0,review to come after i get some new year's eve dinner ;),en-US,1628,2011-03-03,56,12,4.3,20
10000600,16aeeef0420802fdd8af7698d174cfd0,"old habits (wicked lovely, #2.6)",175855,melissa marr,2b46899ffabde0ca95989f645ad1ce8f,5.0,again... this was very entertaining to read. gives you adding insight into the side characters lives and struggles.,en-US,1628,2011-05-08,115,18,4.3,20
10000600,343113e79fe98f5a926ecf9352240262,"old habits (wicked lovely, #2.6)",175855,melissa marr,e586b770c958ac197b55e40b2fe5c927,5.0,irial and niall... what else is there?,en-US,1628,2011-01-25,38,7,4.3,20
10000600,478d726b4274635bcb9c472939f1e9be,"old habits (wicked lovely, #2.6)",175855,melissa marr,0951c5c5edde4755c9c5c3546f38398a,5.0,"this lovely little read arrived on my kindle last night at midnight and since i'm such a big fan of melissa marr's wicked lovely series i just had to read it then and there. i finished at 2 a.m. :) old habits is another wonderful novella taking place after ink exchange and let me say includes my 3 favorite men of the series seth, irial, and niall. where stopping time focused on niall and iri's love for leslie. old habits centers on niall and iri's love for each other and niall accepting his role as the new dark king. i love how melissa explores the depths of these characters and what they will sacrifice for each other. there's more violence in this one i love the battle scene with develin. there's also a steamy scene for iri i won't say with who since my reviews are spoiler fr

# Text Cleaning and Normalization

In [0]:
"""
Define and apply a text cleaning function to preprocess review text. 
The function converts text to lowercase, removes URLs, numbers, punctuation, 
and extra spaces. It also filters out very short reviews and adds 
two new columns for word and character counts before saving the cleaned dataset.
"""

@F.udf(returnType=StringType())
def clean_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"\d+", " num ", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

clean_train = train.withColumn("clean_review", clean_text(F.col("review_text")))
clean_train = clean_train.filter(F.length("clean_review") > 10)

clean_train = clean_train.withColumn("review_length_words", size(split(col("clean_review"), " "))) \
                   .withColumn("review_length_chars", length(col("clean_review")))

display(clean_train.limit(10))

clean_train.write.format("delta").mode("overwrite").save(
    output_base + "/clean_train/"
)


book_id,review_id,title,author_id,author_name,user_id,rating,review_text,language_code,n_votes,date_added,review_length,review_length_in_words,average_rating,number_of_reviews,clean_review,review_length_words,review_length_chars
10001021,16b1077b60e3674e4edaef7951194c43,"dragon ball, vol. 1: son goku (dragon ball, #1)",2686,akira toriyama,ebc700f4f7abb6203ef6f8d987d80e40,4.0,"podem ler a opiniao completa aqui. even though the anime was never one of my favourites, i always wanted to read the manga, and so when this got published in portugal, i went ahead and bought it. i'm glad i did! the drawings are very good, the story is as funny as i remember, and goku's 'innocence' just gets me everytime. i'll definetly keep buying.",por,28,2010-12-23,357,65,4.0,3,podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,65,336
10001021,3c604adc11027e4621d0d1c189e3f573,"dragon ball, vol. 1: son goku (dragon ball, #1)",2686,akira toriyama,873f794f39e4490562e0e85845e1059e,4.0,"lembro-me de ver a serie televisiva quando era mais pequena, na sic, e quando vi a manga a venda mal quis acreditar. :d acho que dei pulos de contente, devo ter gritado um valente ""yay!"" e a slayra deve ter pensado (ou melhor, deve ter tido a certeza) que eu havia ensandecido. infelizmente so via os volumes mais recentes, mas veio a feira e o meu irmao (talvez ainda mais doido que eu pela serie) la fez o favor de comprar. apesar de fa, confesso que pouco me lembro de cada episodio, tenho mais presente a serie dragon ball z, mas parece-me que a manga segue mais ou menos a mesma historia. tem as mesmas piadas, as mesmas personagens, o mesmo desenho... nao sera a 8a maravilha do mundo, mas entretem como os desenhos animados o fizeram em tempos. :p eu nao me lembrava era que havia tanta piada com cuecas... *revira os olhos* vale o dinheiro gasto: talvez nao seja para toda a gente, mas va la, quem nao cresceu maravilhado com as aventuras do son goku? :p",por,28,2011-05-07,966,179,4.0,3,lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,180,925
10003693,b941350cd31579b94d3c00a319477b7a,dark rain: a new orleans story,319345,mat johnson,0d3f29e54c4e31039375e09daab228bb,2.0,"mat johnson's work usually has more nuance and depth than this, but the hurricane katrina elements feel cut-and-pasted onto a mildly intriguing heist plot that's hobbled by one-dimensional characters and awkward pacing, as if an early draft of a hollywood script was turned into a comic book. simon gane's artwork is solid, with expressive characters and effective framing, and lee loughridge's use of gray tones and strategic use of blue work well for the story's waterlogged setting, but it all adds up to a disappointing swing and a miss, the whole far less than the sum of its parts.",en-US,22,2012-08-16,587,99,2.0,1,mat johnsons work usually has more nuance and depth than th

# Extract text-based features


In [0]:
"""
Load the cleaned dataset and compute sentiment scores using NLTK’s VADER analyzer. 
For each review, calculate positive, neutral, negative, and compound sentiment values. 
Display a sample of the results and save the sentiment-enhanced dataset to the Gold layer.
"""

clean_train = spark.read.format("delta").load(
    output_base + "/clean_train/"
)

nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_pos(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['pos']

def get_sentiment_neu(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neu']

def get_sentiment_neg(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neg']

def get_sentiment_compound(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['compound']

from pyspark.sql import functions as F

sentiment_train = (
    clean_train
    .withColumn("sentiment_pos", F.udf(get_sentiment_pos, DoubleType())("clean_review"))
    .withColumn("sentiment_neu", F.udf(get_sentiment_neu, DoubleType())("clean_review"))
    .withColumn("sentiment_neg", F.udf(get_sentiment_neg, DoubleType())("clean_review"))
    .withColumn("sentiment_compound", F.udf(get_sentiment_compound, DoubleType())("clean_review"))
)

display(
    sentiment_train.select(
        "clean_review",
        "sentiment_pos",
        "sentiment_neu",
        "sentiment_neg",
        "sentiment_compound"
    ).limit(10)
)

sentiment_train.write.format("delta").mode("overwrite").save(
    output_base + "sentiment_train/"
)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


clean_review,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
this takes place after ink exchange also it focuses on nialls and irials relationship niall is dealing with being the new dark king even though he doesnt desire the role he cannot think of anyone else who is more capable of being itbesides irial who refuses it back as niall is settling into his new role as dark king he still struggles with his feelings for irial and leslie irial is working behind nialls back to help keep him safe even though he isnt the king anymore he is willing to put himself at risk to protect niall he is trying to take part of all the really bad things in court to help niall transition niall doesnt want irial to interfere and go against his commands even though i feel he appreciates his help he cannot afford to have anyone go against him if he is to truly take charge of the dark court devlin is the high court queens sister he does her bidding even though she knows one day he will become her enemy his true desires is with the dark court irial has a friendship with the high court queen irial and him have had an intimate relationship from long ago they are both struggling with their feeling for each other i personally think they should end up together i feel that they make a better pairing than with leslie though most faeries live a polyamorous life,0.123,0.809,0.068,0.9276
love melissa marrs wicked lovely books and this novella did not let me down very interesting and compelling story we learn so much more of the inner workings of the fairie regents like irial and sorchas close yet indiffent relationship sorchas relationship with her brother devlin and my favorite the sorid love hate relationship of nial and irial two of my favorite characters together again in this short but must read couldnt put the story down i read it twice i love nial and irial that much,0.224,0.731,0.045,0.9449
review to come after i get some new years eve dinner,0.0,1.0,0.0,0.0
again this was very entertaining to read gives you adding insight into the side characters lives and struggles,0.147,0.738,0.115,0.1761
irial and niall what else is there,0.0,1.0,0.0,0.0
this lovely little read arrived on my kindle last night at midnight and since im such a big fan of melissa marrs wicked lovely series i just had to read it then and there i finished at num am old habits is another wonderful novella taking place after ink exchange and let me say includes my num favorite men of the series seth irial and niall where stopping time focused on niall and iris love for leslie old habits centers on niall and iris love for each other and niall accepting his role as the new dark king i love how melissa explores the depths of these characters and what they will sacrifice for each other theres more violence in this one i love the battle scene with develin theres also a steamy scene for iri i wont say with who since my reviews are spoiler free so if youre a fan of the wicked lovely series old habits is a must read num stars,0.24,0.684,0.077,0.9868
old habits is a short novella of ten chapters that focuses on the relationship between irial the former dark king and niall the new king of the dark court it is set between the events taking place in ink exchange and fragile eternity irial and niall have had a long and rocky history irial who is now serving as nialls adviser and protector must learn to take a subservient role now that niall is beginning to accept his status as the new king its not an easy adjustment for either and in trying to protect niall irial makes some decisions behind the kings back that do not sit well with the new ruler this short story expands upon this transitional period as roles become reversed this is another incredible short story by author melissa marr that allows fans of this series to get one more look at two of their favorite characters irial and niall in stopping time we got to see these two dealing with each other and their feelings for leslie and in this 

### TF-IDF Features

In [0]:
"""
Load the sentiment-enhanced dataset and extract TF-IDF features using a Spark ML pipeline. 
The process includes tokenization, stop word removal, term frequency calculation, 
and inverse document frequency weighting. Cache the result, preview the output, 
and save the TF-IDF feature dataset to the Gold layer.
"""

sentiment_train = spark.read.format("delta").load(
     output_base + "sentiment_train/"
)

tokenizer = Tokenizer(inputCol="clean_review", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

cv = CountVectorizer(
    inputCol="filtered_words",
    outputCol="raw_features",
    vocabSize=5000
)

idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features"
)

pipeline = Pipeline(stages=[tokenizer, remover, cv, idf])
tfidf_model = pipeline.fit(sentiment_train)
tfidf_train = tfidf_model.transform(sentiment_train)

tfidf_train.cache()

display(tfidf_train.select("clean_review", "tfidf_features").limit(10))

tfidf_train.write.format("delta").mode("overwrite").save(
     output_base + "/tfidf_train/"
)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

🏃 View run bemused-squirrel-380 at: https://adb-664380929050065.5.azuredatabricks.net/ml/experiments/1597625832640364/runs/889f8da0f6f346df90b605ed36b51e56
🧪 View experiment at: https://adb-664380929050065.5.azuredatabricks.net/ml/experiments/1597625832640364


clean_review,tfidf_features
ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh,"Map(vectorType -> sparse, length -> 5000, indices -> List(1, 382, 1531, 3585), values -> List(1.2222949860555519, 4.449505299476855, 5.388105144834655, 6.25235711466875))"
absorbing cheesy oddly modern in its treatment of indians while focusing as one would expect on the lives of europeans and americans in colonial india at some point between the world wars the characters are satisfyingly rounded out though bromfield relies a bit too heavily on providing an explicit transcript of their thoughts in a shifting close third person found myself intrigued at the end after a bit of a lull in the middle worthwhile period piece,"Map(vectorType -> sparse, length -> 5000, indices -> List(4, 8, 32, 38, 44, 57, 66, 137, 172, 208, 424, 441, 478, 483, 489, 653, 849, 864, 1175, 1939, 2344, 2469, 2473, 2477, 2766, 3015, 3297, 3524, 3812, 3994, 4432), values -> List(1.198366434319084, 1.5741593963057414, 2.1149752958307846, 2.3558440985013354, 2.303563948687721, 4.776316636862713, 2.429154753407576, 2.9676486354282807, 3.1756743474188687, 3.268047667549884, 3.8292984330038946, 3.8486199623122017, 3.8976090861922494, 3.920915800413851, 3.9733990513298747, 4.2849607496570625, 4.478434700903878, 4.513976658849338, 4.7647918267828056, 5.556968733742572, 5.6253745348179605, 5.653967075377774, 6.038182093932916, 5.599347121254717, 5.911528533790087, 5.825636597564066, 5.924264737845336, 6.019926071584103, 6.110629231159512, 6.211204033764415, 6.30959290094094))"
a very easy read book with the extract of num fundamental questions existentialism and justice funny that i just finished another book course about death from kagan death open yale courses and i could totally understand the dualism about life consciousness physicalismetc the second part was coincidentally matched to another open course i have watched years back then justice whats the right thing to do talking about what is right from wrong how do we define morality should the goodness be confined within people we care not people in general how about inequality of wealth therefore this book is like a conclusion of these num famous open courses well written simple and clear,"Map(vectorType -> sparse, length -> 5000, indices -> List(0, 1, 2, 5, 19, 20, 34, 49, 73, 78, 81, 86, 88, 90, 139, 201, 205, 236, 246, 253, 279, 306, 344, 364, 460, 515, 521, 542, 626, 654, 698, 795, 890, 1262, 1278, 1822, 1895, 2147, 3456, 4308, 4692), values -> List(1.9025307511649303, 2.4445899721111037, 0.9593314809609421, 1.3587115539550743, 2.020481111377068, 1.8610726921671523, 4.581003685900576, 2.366634396426242, 5.030027212950598, 2.6399138877857538, 2.536540081935406, 5.274569064761883, 2.6810921900996716, 2.726736424477252, 2.948130865898204, 3.1869472266838623, 3.2409266546507864, 6.600316878156822, 3.349304441376167, 7.014124127516886, 3.502193992495894, 3.5244815308118937, 3.5754084138591047, 3.7026943658675724, 3.933556051617472, 4.055682871416399, 4.008803619024791, 4.025054317952937, 4.174338247954137, 12.653053010220166, 4.291527046482744, 4.40290065611984, 4.478224691321419, 4.930864365384597, 10.0047888317099, 5.287574688142034, 5.323870265670888, 5.443397461130517, 6.079037917820448, 6.439689269332383, 6.479085420100311))"
first published in num very short but interesting introduction to the great philosophical questions on free will morality justice knowledge his newest book mind and cosmos shows he has basically held true to his stance on his positions over the years,"Map(vectorType -> sparse, length -> 5000, indices -> List(0, 1, 13, 24, 58, 90, 145, 203, 224, 414, 504, 515, 751, 836, 1001, 1041, 1118, 1278, 3071, 4692, 4711), values -> List(0.6341769170549768, 1.2222949860555519, 1.7403513272147784, 1.9011377111028245, 2.3396302950803554, 2.726736424477252, 3.0052145652889704, 3.2248076006618174, 3.2802479098170036, 3.808850210621284, 3.975714932002646, 4.055682871416399, 4.358464770151816, 4.457559068657704, 4

### BERT embedding

In [0]:
"""
Load the TF-IDF dataset and generate semantic embeddings using the SentenceTransformer model. 
Each cleaned review is converted into a BERT embedding represented as a list of floats. 
The embeddings are added as a new column and the resulting dataset is saved to the Gold layer.
"""

tfidf_train = spark.read.format("delta").load(
     output_base + "/tfidf_train/"
)

model_name = "all-MiniLM-L6-v2"

def embed_text(text):
    """
    Given a cleaned review string, return its BERT embedding as a list of floats.
    This function will be wrapped as a Spark UDF to apply on each row in parallel.
    """
    if not text:
        return None
    global model
    try:
        model
    except NameError:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer(model_name)
    return model.encode(text).tolist()

embed_udf = udf(embed_text, ArrayType(FloatType()))
embedded_train = tfidf_train.withColumn("bert_embedding", embed_udf("clean_review"))

display(embedded_train.select("clean_review", "bert_embedding").limit(10))

embedded_train.write.format("delta").mode("overwrite").save(
     output_base + "/embedded_train/"
)


clean_review bert_embedding ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh List(-0.08079613, 0.017699823, -0.024177115, -0.031205475, -0.024723426, 0.038648084, -0.002068807, 0.11385411, 0.018964663, -0.023509882, 0.05500668, -0.038408954, 0.050342713, 0.011949615, -0.066371374, 0.04065127, -0.09360394, 0.0029636684, -0.028106365, -0.050657503, -0.10755427, 0.09619293, -0.031027023, -0.0032616537, -0.025974697, -0.019466242, 0.0483675, 0.014060312, 0.030750096, -0.113610476, 0.025702024, 0.10459554, 0.029868484, 0.015543847, -0.03637747, -0.007649204, -0.08119362, -0.025276193, 0.037412427, 0.0018053196, -0.02461513, -0.055968992, 0.014714644, 0.0081101665, -0.031470228, 0.08185737, -0.046558373, 0.01444032, -0.015328355, 0.09805616, -0.075251594, -0.03139782, -0.0439971, 0.052152447, 0.048269074, -0.058717385, -0.033708487, 0.029233895, -0.050789632, -0.038037904, -0.039793864, 0.059523754, -0.05279839, -0.042949334, 0.064788714, 0.053843632, -0.02628834, -0.004960689, -0.034807533, 0.051446248, -0.019882651, -0.07814618, -0.017575605, 0.083832964, -0.080934234, -0.022830829, -0.02546959, -0.034926273, 0.08112198, 0.035292063, -0.08009618, -0.03451225, -0.06413994, 0.011419758, -0.05303851, 0.020309627, -0.10160368, 0.05071485, -0.06382282, -0.05127363, 0.103767134, -0.07619538, -0.052220076, -0.010037083, -0.036054395, -0.017799994, 0.012218311, 0.012336027, 0.0050604325, 0.054008737, -0.0033229897, 0.066001, -0.019464763, 0.04452569, -0.10282349, -0.07056175, 0.067537434, 0.00476042, 0.019662404, -0.011989323, 0.019713668, -0.011276408, -0.004044942, -0.057796326, 0.05362825, -0.065060824, 0.005964639, 0.02738062, -0.0029461782, -5.795227E-4, -0.010448573, 0.040789302, -0.03602345, -0.036590934, -0.0193285, -0.016364228, 0.025868308, 1.8085069E-32, 0.004811921, -0.04595397, -0.012155988, 0.049652897, 0.023890018, -0.009922454, -0.003541668, -0.010809003, 0.02590373, -0.023682116, -0.014543337, -0.06143648, -0.020319676, -0.09921491, -0.017576285, -0.096126474, -0.05740058, -0.024186585, -0.11147677, 0.06444026, -0.02227246, -0.009582761, 0.049791057, 0.0014254337, -0.046677217, -0.016499542, 0.031864014, -0.04551383, 0.11866444, 0.026224658, 0.00827227, -0.005391479, -0.12669241, -0.073383994, -0.042876657, 0.015474328, -0.07636005, -0.015624152, 0.036841027, -0.05492568, -0.019782431, 0.03282236, 0.006280812, 0.033042066, 0.022620624, 0.08987182, -0.0033345318, 0.02897855, -0.05677385, -0.054472126, -0.04901525, 0.0026616044, -0.05391692, -0.0616687, 0.13082373, 0.014800038, 0.003437901, 0.022930508, -0.02490785, 0.124453835, 0.026282081, 0.09389294, 0.0012509677, 0.036212977, -0.015770873, -0.08656737, -0.04682532, -0.101088844, -0.02351964, -0.0048141843, 0.01625858, -0.03179165, 0.19051126, 0.0482281, -0.052978326, 0.022462986, 0.059688337, 0.02568797, -0.06357025, -0.055848736, -0.0057763197, -0.004582313, -0.047700323, -0.04497573, 0.0073796688, -0.08516401, 0.002410631, -0.15175372, -0.016118709, 0.0023849977, -0.002858886, 0.024928732, 0.0039317664, 0.02006771, -0.039080843, -1.6664885E-32, 0.008342944, -0.022268493, -0.013125661, 0.052263524, 0.0026979097, -0.018744744, 0.030974157, -0.011409046, 0.12252527, 0.01564005, 0.036262244, 0.04841123, 0.002632409, 0.005714329, 0.041622806, -0.038509183, 0.10635395, 0.044015847, -0.05247164, 0.033265643, 3.9684077E-4, 0.13195385, -0.024898153, 0.002319436, -0.008694633, 0.10777771, -0.09351921, 0.027246043, -0.037890457, 0.026297463, 0.07141122, -0.07051379, -0.056982003, -0.0013729926, -0.06291526, -0.020135514, 0.017415954, 0.071416296, -0.03668231, -0.028401718, 0.061766095, 0.024807649, -0.06329992, -0.030400787, 0.020741342, -0.030295735, -0.05866319, -0.039411303, 0.072957925, -0.020399025, 0.056171957, 0.028300654, -0.011559022, -0.027469428, 0.005595343, 0.008716368, 0.0020860336, -0.08845645, 0.027035931, 0.0057603614, -0.04254592, -8.169932E-4, 0.0062966947, 0.048674732, -0.07470319, -0.021374503, -0.011574421, 0.027372897, 0.017616453, 0.0043321266,

### Additional features

In [0]:
"""
Compute the readability score for each review using the Flesch Reading Ease metric from the textstat library. 
Higher scores indicate simpler and easier-to-read text. The result is added as a new column and a sample is displayed.
"""

embedded_train = spark.read.format("delta").load(
    output_base + "/embedded_train/"
)

def compute_readability(text):
    """
    Compute Flesch Reading Ease score for a given review.
    Returns 0.0 if the text is empty or invalid.
    """
    if not text:
        return 0.0
    try:
        return float(textstat.flesch_reading_ease(text))
    except:
        return 0.0

readability_udf = udf(compute_readability, DoubleType())

embedded_train = embedded_train.withColumn(
    "readability_score", readability_udf("clean_review")
)

display(embedded_train.select("clean_review", "readability_score").limit(5))


clean_review,readability_score
ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh,95.93928571428573
absorbing cheesy oddly modern in its treatment of indians while focusing as one would expect on the lives of europeans and americans in colonial india at some point between the world wars the characters are satisfyingly rounded out though bromfield relies a bit too heavily on providing an explicit transcript of their thoughts in a shifting close third person found myself intrigued at the end after a bit of a lull in the middle worthwhile period piece,-8.657662337662316
a very easy read book with the extract of num fundamental questions existentialism and justice funny that i just finished another book course about death from kagan death open yale courses and i could totally understand the dualism about life consciousness physicalismetc the second part was coincidentally matched to another open course i have watched years back then justice whats the right thing to do talking about what is right from wrong how do we define morality should the goodness be confined within people we care not people in general how about inequality of wealth therefore this book is like a conclusion of these num famous open courses well written simple and clear,-44.867079646017686
first published in num very short but interesting introduction to the great philosophical questions on free will morality justice knowledge his newest book mind and cosmos shows he has basically held true to his stance on his positions over the years,31.09804878048783
i was looking for an introduction to philosophy but for me that means more than just the questions involved but some reference to philosophers i understand intellectually why the author didnt involve actual philosophers but it meant that this book was of little use to me because my studies have taken me far enough that this was a wee bit too introductory in nature,5.721875000000011


In [0]:
"""
Compute the subjectivity score for each review using TextBlob sentiment analysis. 
Values range from 0 (objective and factual) to 1 (highly subjective or opinion-based). 
The score is added as a new column and a sample of results is displayed.
"""

def compute_subjectivity(text):
    """
    Compute subjectivity score using TextBlob.
    Returns 0.0 for empty or invalid text.
    """
    if not text:
        return 0.0
    return float(TextBlob(text).sentiment.subjectivity)

subjectivity_udf = udf(compute_subjectivity, DoubleType())

embedded_train = embedded_train.withColumn(
    "subjectivity", subjectivity_udf("clean_review")
)

display(embedded_train.select("clean_review", "subjectivity").limit(5))


clean_review,subjectivity
ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh,0.0
absorbing cheesy oddly modern in its treatment of indians while focusing as one would expect on the lives of europeans and americans in colonial india at some point between the world wars the characters are satisfyingly rounded out though bromfield relies a bit too heavily on providing an explicit transcript of their thoughts in a shifting close third person found myself intrigued at the end after a bit of a lull in the middle worthwhile period piece,0.53125
a very easy read book with the extract of num fundamental questions existentialism and justice funny that i just finished another book course about death from kagan death open yale courses and i could totally understand the dualism about life consciousness physicalismetc the second part was coincidentally matched to another open course i have watched years back then justice whats the right thing to do talking about what is right from wrong how do we define morality should the goodness be confined within people we care not people in general how about inequality of wealth therefore this book is like a conclusion of these num famous open courses well written simple and clear,0.5641269841269841
first published in num very short but interesting introduction to the great philosophical questions on free will morality justice knowledge his newest book mind and cosmos shows he has basically held true to his stance on his positions over the years,0.48904761904761906
i was looking for an introduction to philosophy but for me that means more than just the questions involved but some reference to philosophers i understand intellectually why the author didnt involve actual philosophers but it meant that this book was of little use to me because my studies have taken me far enough that this was a wee bit too introductory in nature,0.5


In [0]:
"""
Calculate the average word length for each review to estimate language complexity. 
Longer words suggest more complex writing, while shorter words indicate simpler text. 
The computed values are added as a new column and a sample is displayed.
"""

def compute_avg_word_length(text):
    """
    Calculate average word length for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    return float(np.mean([len(w) for w in words])) if words else 0.0

avg_word_len_udf = udf(compute_avg_word_length, DoubleType())

embedded_train = embedded_train.withColumn(
    "avg_word_length", avg_word_len_udf("clean_review")
)

display(embedded_train.select("clean_review", "avg_word_length").limit(5))


clean_review,avg_word_length
ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh,3.2857142857142856
absorbing cheesy oddly modern in its treatment of indians while focusing as one would expect on the lives of europeans and americans in colonial india at some point between the world wars the characters are satisfyingly rounded out though bromfield relies a bit too heavily on providing an explicit transcript of their thoughts in a shifting close third person found myself intrigued at the end after a bit of a lull in the middle worthwhile period piece,4.909090909090909
a very easy read book with the extract of num fundamental questions existentialism and justice funny that i just finished another book course about death from kagan death open yale courses and i could totally understand the dualism about life consciousness physicalismetc the second part was coincidentally matched to another open course i have watched years back then justice whats the right thing to do talking about what is right from wrong how do we define morality should the goodness be confined within people we care not people in general how about inequality of wealth therefore this book is like a conclusion of these num famous open courses well written simple and clear,5.0265486725663715
first published in num very short but interesting introduction to the great philosophical questions on free will morality justice knowledge his newest book mind and cosmos shows he has basically held true to his stance on his positions over the years,5.121951219512195
i was looking for an introduction to philosophy but for me that means more than just the questions involved but some reference to philosophers i understand intellectually why the author didnt involve actual philosophers but it meant that this book was of little use to me because my studies have taken me far enough that this was a wee bit too introductory in nature,4.734375


In [0]:
"""
Compute the lexical diversity of each review to measure vocabulary richness. 
It is calculated as the number of unique words divided by the total number of words. 
Higher values indicate more varied language use. The results are added as a new column and a sample is displayed.
"""

def compute_lexical_diversity(text):
    """
    Compute lexical diversity for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    unique_words = set(words)
    return float(len(unique_words) / len(words)) if words else 0.0

lexical_div_udf = udf(compute_lexical_diversity, DoubleType())

embedded_train = embedded_train.withColumn(
    "lexical_diversity", lexical_div_udf("clean_review")
)

display(embedded_train.select("clean_review", "lexical_diversity").limit(5))


clean_review,lexical_diversity
ktb jyd lft nzr ldrwr drs trykh msr lhdyth m qbl num wm bdh,1.0
absorbing cheesy oddly modern in its treatment of indians while focusing as one would expect on the lives of europeans and americans in colonial india at some point between the world wars the characters are satisfyingly rounded out though bromfield relies a bit too heavily on providing an explicit transcript of their thoughts in a shifting close third person found myself intrigued at the end after a bit of a lull in the middle worthwhile period piece,0.7922077922077922
a very easy read book with the extract of num fundamental questions existentialism and justice funny that i just finished another book course about death from kagan death open yale courses and i could totally understand the dualism about life consciousness physicalismetc the second part was coincidentally matched to another open course i have watched years back then justice whats the right thing to do talking about what is right from wrong how do we define morality should the goodness be confined within people we care not people in general how about inequality of wealth therefore this book is like a conclusion of these num famous open courses well written simple and clear,0.7168141592920354
first published in num very short but interesting introduction to the great philosophical questions on free will morality justice knowledge his newest book mind and cosmos shows he has basically held true to his stance on his positions over the years,0.8780487804878049
i was looking for an introduction to philosophy but for me that means more than just the questions involved but some reference to philosophers i understand intellectually why the author didnt involve actual philosophers but it meant that this book was of little use to me because my studies have taken me far enough that this was a wee bit too introductory in nature,0.765625


# Combined Feature Set and Output


In [0]:
"""
Load the embedded dataset from the Gold layer and save it as a combined dataset. 
This step consolidates all previously engineered features into a single Delta table 
for easier access in later analysis or model training.
"""

embedded_train = spark.read.format("delta").load(
    output_base + "/embedded_train/"
)

combined_train = embedded_train
combined_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(output_base + "/combined_train/")
